# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [10]:
import os
import json
from datetime import datetime
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
# Environment and OpenRouter api

load_dotenv(override=True)
api_key = os.getenv("OPEN_ROUTER_API_KEY")
OPEN_ROUTER_URL = "https://openrouter.ai/api/v1"

if not api_key or len(api_key) < 10:
    print("Set OPEN_ROUTER_API_KEY in .env")
else:
    print("API key Looks good.")

openai = OpenAI(base_url=OPEN_ROUTER_URL, api_key=api_key)

API key Looks good.


In [12]:
# MODEL AND PROMPTS

MODEL_CLAUDE = "anthropic/claude-3.5-haiku"


In [13]:
#PROMPT

SYSTEM_PROMPT = """
You are a helpful technical tutor who answers questions about provided code or technical programming questions, software engineering, 
data science and LLMs. And other related topics to computer science.Give clear, structured explanations. Use markdown when useful (lists, code blocks). 
Do not wrap your whole reply in a code block.If the user asks for today's date or current time, use the get_current_date tool."""

In [14]:
# tools to get the current date and time

get_current_date_spec = {
    "name": "get_current_date",
    "description": "Get the current date and time. Use when the user asks for today's date, current time, or what day it is.",
    "parameters": {
        "type": "object",
        "properties": {},
        "additionalProperties": False,
    },
}



In [15]:
tools = [{"type": "function", "function": get_current_date_spec}]

In [16]:
def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_current_date":
            now = datetime.now().strftime("%A, %B %d, %Y. Time: %H:%M")
            responses.append({
                "role": "tool",
                "content": now,
                "tool_call_id": tool_call.id,
            })
    return responses

In [21]:
def chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        *history,
        {"role": "user", "content": message},
    ]

    date_display = ""
    response = openai.chat.completions.create(
        model=MODEL_CLAUDE,
        messages=messages,
        tools=tools,
    )
    while response.choices[0].finish_reason == "tool_calls":
        msg = response.choices[0].message
        tool_responses = handle_tool_calls(msg)
        for tr in tool_responses:
            if tr.get("content"):
                date_display = tr["content"]
                break
        messages.append(msg)
        messages.extend(tool_responses)
        response = openai.chat.completions.create(
            model=MODEL_CLAUDE,
            messages=messages,
            tools=tools,
        )

    reply_time = datetime.now()
    time_footer = reply_time.strftime("%d %b %Y, %H:%M")
    stream = openai.chat.completions.create(
        model=MODEL_CLAUDE,
        messages=messages,
        stream=True,
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        body = (f"**Date & time:** {date_display}\n\n" + result) if date_display else result
        yield body + f"\n\n— {time_footer}"

In [ ]:
gr.ChatInterface(
    fn=chat,
    type="messages",
    title="Technical Tutor ",
    description="Ask any technical question and get an answer.",
    examples=[
        "What does yield from do in Python?",
        "What is today's date?",
        "Explain attention in transformers in 3 sentences.",
    ],
    flagging_mode="never",
).launch()